** Some imports **

In [3]:
import pandas as pd
import numpy as np

In [23]:
names = ['ind', 'ax', 'ay', 'az', 'class']
df = pd.read_csv("Dataset/1.csv", sep=',', names=names)
del df['ind']

In [24]:
df.head()

,ax,ay,az,class
0,1502,2215,2153,1
1,1667,2072,2047,1
2,1611,1957,1906,1
3,1601,1939,1831,1
4,1643,1965,1879,1
